For this program, you will be coding some of the ideas around decision trees. In particular, you will be looking into splitting the root node for a tree. Data is provided via an input file. **Be sure to document your code.**

**You may not use an packages or "canned" code in your program other than simple function calls to needed math functions or file I/O**

The input file is a csv file named *DecTreeAssign1.dat* and is located in the Data folder on Google.

The first line contains the class name followed by the n feature names. Your program should allow any number of features >= 1. I.e., the number of features should not be hardcoded. Neither should the number of observations.

Each subsequent line is an observation that has the class indicator, "M" or "B", followed by numeric values for each of the n features.

Read the input file and build a data matrix such that the feature values are scaled in the interval [0, 1]. Calculate  the mean value for each feature after they are scaled.

You will now need to do some calculations for splitting the root node.

For the root node, print the following values to the console:
* Number of class "B" observations
* Number of class "M" observations
* Total number of observations
* Gini index for the root
* Entropy for the root

Here is an example console output:
```
Class B observations:   262
Class M observations:   198
Total observations:     460
Root gini:    0.4932
Root entropy: 0.9860
```

For each feature:

Calculate the Gini index and Entropy for each child node if a split is made on the mean value for the feature. Calculate the combined Gini index and Entropy for the child nodes. Child 1 is the < decision, and Child 2 is the >= decision.

Students should write their own Gini and Entropy functions.

Open a csv output file named: *DecTreeAssign1_out.csv*

Write one line to the output file for each feature that contains the following separated by commas:

* Feature name
* Feature mean value after scaling
* Child 1 count for class "B"
* Child 1 count for class "M"
* Child 1 Gini
* Child 1 Entropy
* Child 2 count for class "B"
* Child 2 count for class "M"
* Child 2 Gini
* Child 2 Entropy
* Combined Gini for Child 1 and Child 2
* Combined Entropy for Child 1 and Child 2

For readibility in my output file, all floats written were limited to 4 decimal places.

Here is an example of a possible first 4 lines of the output file (these are not necessarily what you will get, just an example).
```
F00, 0.3382,  311,   32, 0.1692, 0.7290,   46,  180, 0.3242, 0.7290, 0.2308, 0.5592
F01, 0.3240,  254,   52, 0.2821, 0.9658,  103,  160, 0.4765, 0.9658, 0.3720, 0.8001
F02, 0.3329,  313,   30, 0.1596, 0.7112,   44,  182, 0.3136, 0.7112, 0.2208, 0.5404
F03, 0.2169,  324,   41, 0.1994, 0.6385,   33,  171, 0.2712, 0.6385, 0.2252, 0.5541
```

Load the output file into Google Sheets, Excel or some other spreadsheet program. Determine which feature should be chosen to split on at the root using the combined Gini and Entropy criteria. Is it the same for both?

If you are interested, feel free to expand your program to do more. Be sure to document your code.


In [ ]:
'''
Functions and imports for Decision Tree Assignment #1
'''

import  math

'''
this function scale the numeric features into range [0,1]
argument list:
- feature: an array(vector) of the numeric feature
'''
def scale_feature(feature):
  max = float('-inf')
  min = float('inf')
  # first loop find out the max and the min value of feature array
  for i in range(len(feature)):
    if feature[i] > max:
      max = feature[i]
    if feature[i] < min:
      min = feature[i]
  # second loop scale each element of the feature
  for i in range(len(feature)):
    feature[i] = (feature[i] - min)/(max - min)
  return feature


'''
this function calculate the gini index for a node
it takes in the number of different labels as arguments
'''
def gini_index(num_M, num_B):
  if num_M == 0 and num_B == 0:
    return 0
  total = num_B + num_M
  gini = 1 - ((num_B/total)**2 + (num_M/total)**2)
  return gini

'''
this function calculate the entropy for a node
it takes in the number of different labels as arguments
'''
def entropy(num_M, num_B):
  total = num_M + num_B
  if (num_M == 0 or num_B == 0):  #pure node
    entropy = 0
  else:
    entropy = -((num_M/total)*math.log2(num_M/total) + (num_B/total)*math.log2(num_B/total))
  return entropy

'''
this function calculate the mean of a feature
it takes in the feature vector as an argument
'''
def cal_mean(feature):
  sum = 0
  for i in range(len(feature)):
    sum += feature[i]
  return sum/len(feature)

"""
This function calculate the optimal split for one feature
"""
def find_optimal_split(feature, data_matrix):
   # loop through each possible spliting point
   global_minimum_gini = 999
   global_entropy = 0
   feature_index = 999
   for i in range(len(feature)):
     splitting_point = feature[i]
     m_count_cld1 = 0
     m_count_cld2 = 0
     b_count_cld1 = 0
     b_count_cld2 = 0
     for m in range(len(feature)):
       if feature[m] < splitting_point: # belongs to child 1
         if data_matrix[m][0] == 'M':
           m_count_cld1 += 1
         else:
           b_count_cld1 += 1
       else: # child 2 (>= average)
         if data_matrix[m][0] == 'M':
           m_count_cld2 += 1
         else:
           b_count_cld2 += 1
     # calculate necessary index and write into output file
     gini_cld1 = gini_index(b_count_cld1,m_count_cld1)
     gini_cld2 = gini_index(b_count_cld2,m_count_cld2)
     weighted_gini = (b_count_cld1 + m_count_cld1)/len(data_matrix)*gini_cld1 + (b_count_cld2 + m_count_cld2)/len(data_matrix)*gini_cld2
     entropy_cld1 = entropy(b_count_cld1,m_count_cld1)
     entropy_cld2 = entropy(b_count_cld2,m_count_cld2)
     weighted_entropy = (b_count_cld1 + m_count_cld1)/len(data_matrix)*entropy_cld1 + (b_count_cld2 + m_count_cld2)/len(data_matrix)*entropy_cld2
     if weighted_gini < global_minimum_gini:
       global_minimum_gini = weighted_gini
       global_entropy = weighted_entropy
       feature_index = i
   #print("gini is {}, entropy is {}, at {}'s feature, with split point of {}".format(global_minimum_gini, global_entropy, feature_index, feature[i]))
   #print("chl1_m {} child1_b {} child2_m {} child2_b {}".format(m_count_cld1, b_count_cld1, m_count_cld2, b_count_cld2))
   return global_minimum_gini, global_entropy, feature_index

def calculate_performance(actual, predict):
  pass #return the confusion matrix

In [ ]:
'''
Decision Tree Assignment #1
'''

if __name__ == "__main__":
  """
  step 1:Read the input file and build a data matrix such that the feature values are scaled in the interval [0, 1]
  """
  #Data file name
  data_file_name = "DecTreeAssign1.dat"
  data_matrix = []
  # read the file
  with open(data_file_name, 'r') as data_file_ptr:
    for index, item in enumerate(data_file_ptr):
      item_list = item.split(',')
      if index != 0:  #first row of the data is the title
        data_matrix.append(item_list) #this is the data matrix
      else:
        feature_name = item_list
      
  #get rid of the new line character in last feature name
  feature_name[len(feature_name)-1] = "F{}".format(len(feature_name)-2)

  #normalize each column of the matrix
  for i in range(1, len(data_matrix[0])): # loop for each column/feature
    feature_vector = [data_matrix[j][i] for j in range(len(data_matrix))]
    for k in range(len(feature_vector)):
      feature_vector[k] = float(feature_vector[k]) #covert to float
    feature_vector = scale_feature(feature_vector)
    for m in range(len(data_matrix)):
      data_matrix[m][i] = feature_vector[m]

  """
  step 2: print necessary info for root node
  """
  b_count = 0
  m_count = 0
  for i in range(len(data_matrix)):
    if data_matrix[i][0] == 'B':
      b_count += 1
    else:
      m_count += 1
  print("Class B observations: " + str(b_count))
  print("Class M observations: " + str(m_count))
  print("Total observations: " + str(len(data_matrix)))
  print("Root gini: " + str(gini_index(b_count,m_count)))
  print("Root entropy: " + str(entropy(b_count,m_count)))   
  print("\n\n")

  """
  step 3:
  For each feature: Calculate the Gini index and Entropy for each child node if a split 
  is made on the mean value for the feature. 
  Calculate the combined Gini index and Entropy for the child nodes. 
  Child 1 is the < decision, and Child 2 is the >= decision.
  """

  mean_feature = []
  # initialize the output file
  out_file_name = "DecTreeAssign1_out.csv"
  with open(out_file_name, 'w') as out_file_ptr:
    for i in range(1, len(data_matrix[0])): # loop for each column/feature
      # extract the feature out
      feature_vector = [data_matrix[j][i] for j in range(len(data_matrix))]
      for k in range(len(feature_vector)):
        feature_vector[k] = float(feature_vector[k])
      average = cal_mean(feature_vector)

      m_count_cld1 = 0
      m_count_cld2 = 0
      b_count_cld1 = 0
      b_count_cld2 = 0
      # split the feature based on mean
      for m in range(len(feature_vector)):
        if feature_vector[m] < average: # belongs to child 1
          if data_matrix[m][0] == 'M':
            m_count_cld1 += 1
          else:
            b_count_cld1 += 1
        else: # child 2 (>= average)
          if data_matrix[m][0] == 'M':
            m_count_cld2 += 1
          else:
            b_count_cld2 += 1
      # calculate necessary index and write into output file
      gini_cld1 = gini_index(b_count_cld1,m_count_cld1)
      entropy_cld1 = entropy(b_count_cld1,m_count_cld1)
      gini_cld2 = gini_index(b_count_cld2,m_count_cld2)
      entropy_cld2 = entropy(b_count_cld2,m_count_cld2)
      weighted_gini = (b_count_cld1 + m_count_cld1)/len(data_matrix)*gini_cld1 + (b_count_cld2 + m_count_cld2)/len(data_matrix)*gini_cld2
      weighted_entropy = (b_count_cld1 + m_count_cld1)/len(data_matrix)*entropy_cld1 + (b_count_cld2 + m_count_cld2)/len(data_matrix)*entropy_cld2
      out_file_ptr.write("{}, {:0.4f}, {}, {}, {:0.4f}, {:0.4f}, {}, {}, {:0.4f}, {:0.4f}, {:0.4f}, {:0.4f}\n".format(feature_name[i], average, b_count_cld1, m_count_cld1, gini_cld1, entropy_cld1, b_count_cld2, m_count_cld2, gini_cld2, entropy_cld2, weighted_gini, weighted_entropy))


  """
  for item in data_matrix:
    print(item)
  """
  
  

Class B observations: 357
Class M observations: 212
Total observations: 569
Root gini: 0.4675300607546925
Root entropy: 0.9526351224018599





Optimal Split

In [ ]:
for i in range(1, len(data_matrix[0])): # loop for each column/feature
  # extract the feature out
  feature_vector = [data_matrix[j][i] for j in range(len(data_matrix))]
  glob_gini, glob_entropy, glob_index = find_optimal_split(feature_vector, data_matrix)
  print("for feature {}, combined_gini is {}, combined_entropy is {}, with {}'s feature as splitting point with value of {}".format(feature_name[i], glob_gini, glob_entropy, glob_index, data_matrix[index][i]))

for feature F00, combined_gini is 0.19242486589915644, combined_entropy is 0.4896488694028093, with 514's feature as splitting point with value of 0.3818921860949407
for feature F01, combined_gini is 0.3690697283638583, combined_entropy is 0.7961722035638179, with 131's feature as splitting point with value of 0.3165370307744335
for feature F02, combined_gini is 0.1916093835208614, combined_entropy is 0.4860075931192521, with 205's feature as splitting point with value of 0.3694976159214982
for feature F03, combined_gini is 0.1867183670721237, combined_entropy is 0.4778349162125473, with 38's feature as splitting point with value of 0.23686108165429479
for feature F04, combined_gini is 0.41023756912945447, combined_entropy is 0.8555307306582929, with 444's feature as splitting point with value of 0.3567753001715266
for feature F05, combined_gini is 0.30446499363426294, combined_entropy is 0.6855696883147584, with 23's feature as splitting point with value of 0.20425127292804127
for fea